In [1]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
import fitz
import os
from transformers import BertTokenizer, BertModel
import torch
import faiss
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from llama_index.llms.groq import Groq
import re
import yaml

c:\Users\Monster\Desktop\EngeAI\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Monster\Desktop\EngeAI\.venv\Lib\site-packages\threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Monster\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
with open('auth.yaml', 'r') as config_file:
    config = yaml.load(config_file,Loader=yaml.Loader)
my_api_key = config['api_key']

In [4]:
def extract_text_from_pdfs(pdf_folder):
    """
    Extracts text from all PDF files in a specified folder.

    Args:
        pdf_folder (str): The path to the folder containing PDF files.

    Returns:
        list: A list of strings, where each string is the text extracted from the PDF files.
    """
    texts = []  # Initialize an empty list to store extracted texts
    
    # Loop through all files in the specified folder
    for pdf_file in os.listdir(pdf_folder):
        # Check if the current file is a PDF
        if pdf_file.endswith('.pdf'):
            # Open the PDF file
            doc = fitz.open(os.path.join(pdf_folder, pdf_file))
            
            # Loop through all pages in the PDF file
            for page_num in range(len(doc)):
                # Load the current page
                page = doc.load_page(page_num)
                
                # Extract text from the current page
                text = page.get_text()

                cleaned_text = re.sub(r'\.{4,}', '.', text)

                
                # Append the extracted text to the list
                texts.append(cleaned_text)
    
    return texts  # Return the list of extracted texts

# Specify the folder containing the PDF files
# Use os.path.join to construct a relative path
# This assumes the 'EngeAI' folder is in the same directory as your script
current_directory = os.path.dirname(os.getcwd())  # Get the directory of the current script
pdf_folder = os.path.join(current_directory, 'EngeAI','files')

# Call the function and store the extracted texts in a variable
pdf_texts = extract_text_from_pdfs(pdf_folder)

In [5]:
pdf_texts[2]

'Informationen für Ihren Versicherungsvertrag\nSeite 1 von 3\nBASIS_PACK_WB\nWB/D/1006/XIII/03/22\n1\nWer ist Ihr Vertragspartner\nVersicherer ist die Standard Life International DAC (90 St Stephens Green, Dublin 2, Irland, \nRegister-Nr. 408507). Die Anschrift der für Sie zuständigen Zweigniederlassung lautet: \nStandard Life Versicherung\nZweigniederlassung Deutschland der\nStandard Life International DAC\nLyoner Straße 9\n60528 Frankfurt/Main\nLadungsfähige Anschrift und Sitz der Zweigniederlassung\nStandard Life Versicherung \nZweigniederlassung Deutschland der \nStandard Life International DAC\nLyoner Straße 9\n60528 Frankfurt\nDie Zweigniederlassung ist eingetragen beim Amtsgericht Frankfurt am Main unter der Registernummer \nHRB 111481.\nVertreter der Zweigniederlassung und zugleich Hauptbevollmächtigter: Richard Reinhard.\nStandard Life International DAC ist eine irische Versicherungsgesellschaft mit Sitz in Dublin und gehört zur \nPhoenix Gruppe in Großbritannien. Standard Lif

In [6]:
from nltk.tokenize import word_tokenize

# Function to segment texts with overlapping chunks
def segment_text_with_overlap(texts, chunk_size, chunk_overlap):
    segments = []  # List to store the resulting text segments
    for text in texts:
        words = word_tokenize(text)  # Tokenize the text into words
        step = chunk_size - chunk_overlap  # Calculate the step size for the overlapping chunks
        for i in range(0, len(words), step):  # Loop over the words with the calculated step size
            chunk = ' '.join(words[i:i+chunk_size])  # Create a chunk of the specified size
            segments.append(chunk)  # Add the chunk to the list of segments
    return segments  # Return the list of text segments

# Example usage
segments = segment_text_with_overlap(pdf_texts, chunk_size=100, chunk_overlap=5)

# Print each segment with its index
print(f"Segment {2}:", segments[2])

Segment 2: Informationen für Ihren Versicherungsvertrag Seite 1 von 3 BASIS_PACK_WB WB/D/1006/XIII/03/22 1 Wer ist Ihr Vertragspartner Versicherer ist die Standard Life International DAC ( 90 St Stephens Green , Dublin 2 , Irland , Register-Nr . 408507 ) . Die Anschrift der für Sie zuständigen Zweigniederlassung lautet : Standard Life Versicherung Zweigniederlassung Deutschland der Standard Life International DAC Lyoner Straße 9 60528 Frankfurt/Main Ladungsfähige Anschrift und Sitz der Zweigniederlassung Standard Life Versicherung Zweigniederlassung Deutschland der Standard Life International DAC Lyoner Straße 9 60528 Frankfurt Die Zweigniederlassung ist eingetragen beim Amtsgericht Frankfurt am Main unter der Registernummer HRB 111481 . Vertreter der


In [7]:
# Load the BERT tokenizer and model from the pretrained 'bert-base-multilingual-cased' model
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained('bert-base-multilingual-cased')

# Function to vectorize a list of texts using the BERT model
def vectorize_texts(texts, tokenizer, model):
    # - Tokenize the input texts
    # - return_tensors='pt' converts the tokens to PyTorch tensors
    # - padding=True ensures all sequences are padded to the same length
    # - truncation=True truncates sequences longer than max_length
    # - max_length=512 sets the maximum length of sequences
    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=512)
    
    # Use the model to get the outputs without calculating gradients (disables backpropagation)
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Extract the embeddings from the outputs
    # - last_hidden_state contains the hidden states of the model
    # - [:, 0, :] extracts the embeddings of the [CLS] token (first token)
    embeddings = outputs.last_hidden_state[:, 0, :]
    
    # Return the embeddings
    return embeddings

# Vectorize the list of text segments using the tokenizer and model
vectors = vectorize_texts(segments, tokenizer, model)


In [8]:
# Convert the PyTorch tensor of vectors to a NumPy array
vectors_np = vectors.numpy()

# Get the dimension of the vectors (number of features)
dimension = vectors_np.shape[1]

# Create a FAISS index for fast nearest neighbor search using L2 (Euclidean) distance
index = faiss.IndexFlatL2(dimension)

# Add the vector representations to the FAISS index
index.add(vectors_np)


In [9]:
def retrieve_passages(query, tokenizer, model, index, segments, top_k=5):
    # Tokenize the query
    # - return_tensors='pt' converts the tokens to PyTorch tensors
    # - padding=True ensures the sequence is padded to the same length
    # - truncation=True truncates sequences longer than max_length
    # - max_length=512 sets the maximum length of the sequence
    query_inputs = tokenizer(query, return_tensors='pt', padding=True, truncation=True, max_length=512)
    
    # Use the model to get the outputs for the query without calculating gradients (disables backpropagation)
    with torch.no_grad():
        query_outputs = model(**query_inputs)
    
    # Extract the embedding for the [CLS] token (first token)
    query_vector = query_outputs.last_hidden_state[:, 0, :].numpy()
    
    # Search the FAISS index for the top_k most similar vectors to the query vector
    distances, indices = index.search(query_vector, top_k)
    
    # Retrieve the corresponding segments based on the indices of the search results
    results = [segments[idx] for idx in indices[0]]
    
    # Return the most relevant segments and their corresponding distances
    return results, distances

# Example query
query = "Dieser Wert ist inhaltsleer und nicht interpretierbar. Aus ihm ist also weder ein Rückschluss auf eine Religionszugehörigkeit oder Nichtreligionszugehörigkeit noch ein Rückschluss auf einen vorliegenden oder nicht vorliegenden Sperrvermerk möglich."

# Retrieve the relevant passages and their distances from the query
relevant_passages, my_distance = retrieve_passages(query, tokenizer, model, index, segments)

# Print the relevant passages and distances
for i, (passage, distance) in enumerate(zip(relevant_passages, my_distance[0])):
    print(f"Passage {i+1} (Distance: {distance}): {passage}")

Passage 1 (Distance: 18.46746063232422): Anspruch darauf , dass wir den in c beschriebenen Wert der zum Todeszeitpunkt berechneten → Todesfallleistung auszahlen . e ) Der Anspruch auf Zahlung im Todesfall wird fällig , wenn der Monat abläuft , in dem die → versicherte Person stirbt , und wenn uns alle erforderlichen Dokumente vorliegen . Mehr zu den erforderlichen Dokumenten im Todesfall finden Sie in 6.1 . 2.5 Erhalten Sie eine Überschussbeteiligung ? Dem → Versicherungsnehmer steht eine Beteiligung an den Überschüssen und Bewertungsreserven von Standard Life International DAC im Sinne von § 153 , Abs . 1 VVG ( Versicherungsvertragsgesetz ) nicht zu .
Passage 2 (Distance: 18.54389762878418): den erforderlichen Doku- menten im Todesfall finden Sie in 6.1 . Stirbt die versicherte Person nach dem Rentenbeginndatum , wird grundsätzlich keine → Todesfallleis- tung fällig . Optional kann Kapitalschutz vereinbart werden ( siehe 2.6 ) . Ob Kapitalschutz vereinbart ist , entnehmen Sie bitte Ih

In [10]:
def create_response(passages):
    # Join the passages with double newlines to create the response text
    response = "\n\n".join(passages)
    return response

# Print the response created from the relevant passages
print(create_response(relevant_passages))


Anspruch darauf , dass wir den in c beschriebenen Wert der zum Todeszeitpunkt berechneten → Todesfallleistung auszahlen . e ) Der Anspruch auf Zahlung im Todesfall wird fällig , wenn der Monat abläuft , in dem die → versicherte Person stirbt , und wenn uns alle erforderlichen Dokumente vorliegen . Mehr zu den erforderlichen Dokumenten im Todesfall finden Sie in 6.1 . 2.5 Erhalten Sie eine Überschussbeteiligung ? Dem → Versicherungsnehmer steht eine Beteiligung an den Überschüssen und Bewertungsreserven von Standard Life International DAC im Sinne von § 153 , Abs . 1 VVG ( Versicherungsvertragsgesetz ) nicht zu .

den erforderlichen Doku- menten im Todesfall finden Sie in 6.1 . Stirbt die versicherte Person nach dem Rentenbeginndatum , wird grundsätzlich keine → Todesfallleis- tung fällig . Optional kann Kapitalschutz vereinbart werden ( siehe 2.6 ) . Ob Kapitalschutz vereinbart ist , entnehmen Sie bitte Ihrem Versicherungsschein .

. Aus ihm ist also weder ein Rückschluss auf eine Reli

The LLM Model

In [11]:
def generate_answer(question):
    #vectorize_q = vectorize_texts(texts=question, tokenizer=tokenizer, model=model)
    relevant_passages,dist = retrieve_passages(query= question , tokenizer=tokenizer, model=model, index=index, segments=segments, top_k=5)
    
    llm = Groq(model="llama3-70b-8192", api_key=my_api_key)

    prompt = f"""   Beantworten Sie diese Frage nur anhand der folgenden Absätze 
                    Bringen Sie keine Informationen von außerhalb der Absätze ein und 
                    umformulieren und leicht verständlich machen
                    {question}{create_response(relevant_passages)}"""
    answer = llm.complete(prompt)
    return answer

# Print the response created from the relevant passages


In [12]:
question ="""Welche Fonds bietet die Standard Life International DAC speziell für Vorsorgeprodukte von Standard Life an,
            und welche Vorteile haben diese Fonds im Vergleich zu öffentlich zugelassenen Fonds?"""
print(generate_answer(question))


Die Standard Life International DAC bietet Managed Portfolios an, die speziell für Vorsorgeprodukte von Standard Life konzipiert sind. Diese Fonds sind nicht öffentlich zugelassen und bieten im Vergleich zu öffentlich zugelassenen Fonds den Vorteil, dass sie eine breite Streuung der Anlagen anstreben, um das Risiko zu verringern und den Erfolg der Anlage zu stabilisieren.
